In [3]:
#pip install pororo

2022-11-25 05:33:33 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


In [179]:
from pororo import Pororo
import pandas as pd
import numpy as np

In [180]:
Pororo.available_tasks()

"Available tasks are ['mrc', 'rc', 'qa', 'question_answering', 'machine_reading_comprehension', 'reading_comprehension', 'sentiment', 'sentiment_analysis', 'nli', 'natural_language_inference', 'inference', 'fill', 'fill_in_blank', 'fib', 'para', 'pi', 'cse', 'contextual_subword_embedding', 'similarity', 'sts', 'semantic_textual_similarity', 'sentence_similarity', 'sentvec', 'sentence_embedding', 'sentence_vector', 'se', 'inflection', 'morphological_inflection', 'g2p', 'grapheme_to_phoneme', 'grapheme_to_phoneme_conversion', 'w2v', 'wordvec', 'word2vec', 'word_vector', 'word_embedding', 'tokenize', 'tokenise', 'tokenization', 'tokenisation', 'tok', 'segmentation', 'seg', 'mt', 'machine_translation', 'translation', 'pos', 'tag', 'pos_tagging', 'tagging', 'const', 'constituency', 'constituency_parsing', 'cp', 'pg', 'collocation', 'collocate', 'col', 'word_translation', 'wt', 'summarization', 'summarisation', 'text_summarization', 'text_summarisation', 'summary', 'gec', 'review', 'review_s

In [181]:
ner = Pororo(task="ner", lang="ko")

2022-11-25 08:07:14 | INFO | pororo.models.brainbert.tasks.sequence_tagging | [input] dictionary: 4005 types
2022-11-25 08:07:14 | INFO | pororo.models.brainbert.tasks.sequence_tagging | [label] dictionary: 41 types


In [188]:
from itertools import permutations
temp_list =  ner('Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.')
print(temp_list)

remove_temp_list = [(i[0],i[1])  for i in temp_list if i[1] != 'O' ]
#print(remove_temp_list)

temp = []
for idx , item in enumerate(permutations(['word'], 2)) :
    if item[0][1] == 'PERSON' :
        temp.append(item)

for i in temp :
    print(i)

[('Something', 'ARTIFACT'), ('〉는', 'O'), (' ', 'O'), ('조지 해리슨', 'PERSON'), ('이', 'O'), (' ', 'O'), ('쓰고', 'O'), (' ', 'O'), ('비틀즈', 'PERSON'), ('가', 'O'), (' ', 'O'), ('1969년', 'DATE'), (' ', 'O'), ('앨범', 'O'), (' ', 'O'), ('《', 'O'), ('Abbey Road', 'ARTIFACT'), ('》에', 'O'), (' ', 'O'), ('담은', 'O'), (' ', 'O'), ('노래다.', 'O')]


In [164]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
from tqdm import tqdm
import pathlib
import pickle
from itertools import combinations , permutations

def num_to_label(label: np.ndarray) -> list:
    """
    숫자로 되어 있던 class를 원본 문자열 라벨로 변환 합니다.
    """
    origin_label = []
    with open("/opt/ml/code/dict_num_to_label.pkl", "rb") as f:
        dict_num_to_label = pickle.load(f)
    for v in label:
        origin_label.append(dict_num_to_label[v])

    return origin_label
class RE_Dataset(torch.utils.data.Dataset):
    """Dataset 구성을 위한 class."""

    def __init__(self, pair_dataset: pd.DataFrame, labels: np.ndarray):
        self.pair_dataset = pair_dataset
        self.labels = labels

    def __getitem__(self, idx: int) -> torch.Tensor:
        item = {
            key: val[idx].clone().detach() for key, val in self.pair_dataset.items()
        }
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self) -> int:
        return len(self.labels)


class BaseDataLoader:
    """BaseLine DataLoader 입니다.

    Args:
        load_data :  (str): 가져올 데이터의 주소입니다.
        tokenizer (AutoTokenizer): 데이터를 토큰화할 토크나이저입니다.

    func :
        1)preprocessing_dataset(pd.DataFrame) : 데이터 전처리를 합니다 return pd.DataFrame
        2)tokenized_dataset(pd.DataFrame, tokenizer: AutoTokenizer) : 전처리된 데이터셋을 tokenized_dataset으로 반환 합니다
        3)get_dataset : train, valid 데이터셋을 토큰화 셋으로 변환한 뒤 RE_Dataset 형태로 반환 합니다
        4)get_test_dataset :  test 데이터셋을 토큰화 셋으로 변환한 뒤 RE_Dataset 형태로 반환 합니다
    """

    def __init__(self, data_path: pathlib.Path, tokenizer: AutoTokenizer):
        self.data_path = data_path
        self.tokenizer = tokenizer

    def preprocessing_dataset(self, dataset: pd.DataFrame) -> pd.DataFrame:
        """처음 불러온 csv 파일을 원하는 형태의 DataFrame으로 변경 시켜줍니다."""
        subject_entity = []
        object_entity = []

        re_idx = []
        start = 0
        sentence = []
        label =[]
        sub_list = ['PERSON' , 'ORGANIZATION']
        obj_list = ['PERSON' , 'LOCATION', 'ORGANIZATION', 'ARTIFACT', 'DATE' , 'CIVILIZATION']

        print(list( set(sub_list) | set(obj_list)))

        for row in dataset[:]['sentence'] :
            try :
                pororo_to_sentence = list(set(ner(row)))
            except :
                continue
            pororo_to_sentence = [(i[0],i[1])  for i in pororo_to_sentence if i[1] in list(set(sub_list) | set(obj_list))]
            for idx , item in enumerate(combinations(pororo_to_sentence, 2)) :
                if (item[0][1] in sub_list) and (item[1][1] in obj_list)  :
                    re_idx.append(start)
                    sentence.append(row)
                    subject_entity.append(item[0][0])
                    object_entity.append(item[1][0])
                    label.append(100)
                    start = start +1

        out_dataset = pd.DataFrame(
            {
                "id": re_idx,
                "sentence": sentence,
                "subject_entity": subject_entity,
                "object_entity": object_entity,
                "label": label,
            }
        )

        return out_dataset

    def tokenized_dataset(
        self, dataset: pd.DataFrame, tokenizer: AutoTokenizer
    ) -> torch.Tensor:
        """tokenizer에 따라 sentence를 tokenizing 합니다."""
        concat_entity = []
        for e01, e02 in zip(dataset["subject_entity"], dataset["object_entity"]):
            temp = ""
            temp = e01 + "[SEP]" + e02
            concat_entity.append(temp)

        tokenized_sentences = tokenizer(
            concat_entity,
            list(dataset["sentence"]),
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=256,
            add_special_tokens=True,
        )
        return tokenized_sentences


    def get_test_dataset(self) -> RE_Dataset:
        """데이터셋을 Trainer에 넣을 수 있도록 처리하여 리턴합니다.

        Args:
            data_path (str): 가져올 데이터의 주소입니다.
            tokenizer (AutoTokenizer): 데이터를 토큰화할 토크나이저입니다.

        Returns:
            pd.DataFrame: _description_
        """
        pd_dataset = pd.read_csv(self.data_path)
        dataset = self.preprocessing_dataset(pd_dataset[:1000])
        pre_sentence = dataset['sentence']
        pre_sub_word = dataset['subject_entity']
        pre_obj_word = dataset['object_entity']

        dataset_label = list(map(int, dataset["label"].values))
        dataset_id = dataset["id"]

        # tokenizing dataset
        dataset_tokens = self.tokenized_dataset(dataset, self.tokenizer)
        # make dataset for pytorch.
        dataset = RE_Dataset(dataset_tokens, dataset_label)
        return dataset_id, dataset, dataset_label , pre_sentence , pre_sub_word , pre_obj_word

def inference(model, tokenized_sent, batch_size, device):
    """
    test dataset을 DataLoader로 만들어 준 후,
    batch_size로 나눠 model이 예측 합니다.
    """
    dataloader = DataLoader(tokenized_sent, batch_size, shuffle=False)
    model.eval()

    output_pred = []
    output_prob = []
    for i, data in enumerate(tqdm(dataloader)):
        with torch.no_grad():
            outputs = model(
                input_ids=data["input_ids"].to(device),
                attention_mask=data["attention_mask"].to(device),
                token_type_ids=data["token_type_ids"].to(device),
            )
        logits = outputs[0]
        prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
        logits = logits.detach().cpu().numpy()
        result = np.argmax(logits, axis=-1)

        output_pred.append(result)
        output_prob.append(prob)

    return (
        np.concatenate(output_pred).tolist(),
        np.concatenate(output_prob, axis=0).tolist(),
    )

def load_dataloader(
    dataloder_type: str, data_path: pathlib.Path, tokenizer: AutoTokenizer
):
    """_summary_

    Args:
        dataloder_type (str) : 가져올 dataloder 클래스 입니다 config 확인
        load_data (pathlib.Path): 가져올 데이터의 주소입니다.
        tokenizer (AutoTokenizer): 데이터를 토큰화할 토크나이저입니다.

    Returns:
        dataloader class : dataloader_type에 맞는 class 반환 합니다
    """
    dataloader_config = {
        "BaseDataLoader": BaseDataLoader(data_path, tokenizer),
    }
    return dataloader_config[dataloder_type]

In [165]:
#모델 로드
import torch

model_name = 'klue/roberta-large'
load_model = '../dataset/best_model/klue_roberta-large/3'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(load_model)
model.to(device)
print('--load model --- ')

--load model --- 


In [166]:
test_id, test_dataset, test_label , pre_sentence, pre_sub_word, pre_obj_word = load_dataloader(
    'BaseDataLoader', '../dataset/train/train.csv', tokenizer
).get_test_dataset()


['PERSON', 'LOCATION', 'CIVILIZATION', 'ARTIFACT', 'DATE', 'ORGANIZATION']


In [167]:
pred_answer, output_prob = inference(
        model, test_dataset, 32, device
    )  # model에서 class 추론

pred_answer = num_to_label(pred_answer)  # 숫자로 된 class를 원래 문자열 라벨로 변환.

## make csv file with predicted answer
#########################################################

# 아래 directory와 columns의 형태는 지켜주시기 바랍니다.
output = pd.DataFrame(
    {
        "id": test_id,
        "sentence" : pre_sentence,
        "sub_word" : pre_sub_word,
        "obj_word" : pre_obj_word,
        "pred_label": pred_answer,
        "probs": output_prob,
    }
)

100%|██████████| 354/354 [02:08<00:00,  2.75it/s]


In [176]:
pd.set_option('display.max_rows', None)
print(len(output))
mask1 = (output.pred_label == 'per:origin')

output_mask1 = output.loc[mask1,:]
for idx, item  in output_mask1[:].iterrows() :
    print(idx, item['sentence'] ,'|' ,item['sub_word'] , '|', item['obj_word'])

11323
1179 1900년, 의화단의 난이 일어나 청나라 조정이 열강에 선전 포고를 했을 때에, 이홍장, 장지동, 유곤일 등 지방 총독은 열강과 “동남상호협정”을 맺고 중앙의 명령을 무시했지만, 성선회가 이것을 정리하였다. | 유곤일 | 청나라
1183 1900년, 의화단의 난이 일어나 청나라 조정이 열강에 선전 포고를 했을 때에, 이홍장, 장지동, 유곤일 등 지방 총독은 열강과 “동남상호협정”을 맺고 중앙의 명령을 무시했지만, 성선회가 이것을 정리하였다. | 이홍장 | 청나라
1782 리사 디 배나는 중국에서 개최된 2007년 FIFA 여자 월드컵에서 노르웨이와의 조별 예선 경기(오스트레일리아는 노르웨이와 1-1 무승부를 기록함)에서 1골, 가나와의 조별 예선 경기(오스트레일리아는 가나에 4-1 승리를 기록함)에서 2골, 브라질과의 8강전 경기(오스트레일리아는 브라질에 2-3으로 패배함)에서 1골을 기록하여 오스트레일리아의 8강 진출에 기여했다. | 오스트레일리아 | 가나
1783 리사 디 배나는 중국에서 개최된 2007년 FIFA 여자 월드컵에서 노르웨이와의 조별 예선 경기(오스트레일리아는 노르웨이와 1-1 무승부를 기록함)에서 1골, 가나와의 조별 예선 경기(오스트레일리아는 가나에 4-1 승리를 기록함)에서 2골, 브라질과의 8강전 경기(오스트레일리아는 브라질에 2-3으로 패배함)에서 1골을 기록하여 오스트레일리아의 8강 진출에 기여했다. | 오스트레일리아 | 리사 디 배나
2076 헌강왕(憲康王, ~ 886년, 재위: 875년 ~ 886년)은 신라의 제49대 왕이다. | 헌강왕 | 신라
2835 바투의 형제 베르케하에서, 킵차크 칸국은 베르크 칸이 바그다드 전투와 칼리프 알 무스타으심을 살해한 것으로 경멸했던 훌라구 칸이 다스린 일 칸국의 친척들과 분쟁에 휩싸인다. | 바투 | 킵차크 칸국
2839 바투의 형제 베르케하에서, 킵차크 칸국은 베르크 칸이 바그다드 전투와 칼리프 알 무스타으심을 살해한 것으로 경멸했던 훌라구 칸이 다스린 일 칸국의 친척들

In [178]:
output_mask1.to_csv('./mask2.csv', encoding='utf-8')